In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout, Input, Flatten, Dense, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard, CSVLogger, LearningRateScheduler

# Check if a GPU is available
if tf.config.list_physical_devices('GPU'):
    # Use the available GPU(s)
    print("Training on GPU...")
    tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True)

# Data augmentation
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=30, shear_range=0.2, zoom_range=0.2, width_shift_range=0.2, height_shift_range=0.2, validation_split=0.2)

# Load the training, validation, and test data
batch_size = 8
train_data = train_datagen.flow_from_directory(r'C:\Users\lenovo\Documents\Driver-Drowsiness-Detection-using-Deep-Learning\prepareddata\Train', target_size=(80, 80), batch_size=batch_size, class_mode='categorical', subset='training')
val_data = train_datagen.flow_from_directory(r'C:\Users\lenovo\Documents\Driver-Drowsiness-Detection-using-Deep-Learning\prepareddata\Train', target_size=(80, 80), batch_size=batch_size, class_mode='categorical', subset='validation')
test_data = ImageDataGenerator(rescale=1./255).flow_from_directory(r'C:\Users\lenovo\Documents\Driver-Drowsiness-Detection-using-Deep-Learning\prepareddata\Test', target_size=(80, 80), batch_size=batch_size, class_mode='categorical')

# Load the InceptionV3 model
base_model = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80, 80, 3)))

# Fine-tune the model
for layer in base_model.layers:
    layer.trainable = False
for layer in base_model.layers[-10:]:
    layer.trainable = True

# Add dense layers
x = base_model.output
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(2, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=output)

# Define the callbacks
checkpoint = ModelCheckpoint(r'C:\Users\lenovo\Documents\Driver-Drowsiness-Detection-using-Deep-Learning\model\model.h5', monitor='val_loss', save_best_only=True, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=7, verbose=1, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1, factor=0.1)
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=False)
csv_logger = CSVLogger('training.log', separator=',', append=False)
def lr_schedule(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

# Compile the model
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 10
history = model.fit(train_data, steps_per_epoch=train_data.samples // batch_size, epochs=epochs, validation_data=val_data, validation_steps=val_data.samples // batch_size, callbacks=[checkpoint, early_stop, reduce_lr, tensorboard, csv_logger, LearningRateScheduler(lr_schedule)])

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_data, verbose=1)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)


Found 62340 images belonging to 2 classes.
Found 15584 images belonging to 2 classes.
Found 20834 images belonging to 2 classes.
Epoch 1/10
1659/7792 [=====>........................] - ETA: 1:04:25 - loss: 0.2595 - accuracy: 0.8943